In [1]:
# !pip install duckdb --upgrade

In [2]:
# First cell - Imports and setup
import duckdb
import pandas as pd
import re
import time
from pathlib import Path
from benchmark_utils import BenchmarkResult

print(f"DuckDB version: {duckdb.__version__}")

# Initialize connection and load extensions
conn = duckdb.connect()
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")
conn.execute("INSTALL iceberg;")
conn.execute("LOAD iceberg;")

print("\nExtensions loaded successfully")



DuckDB version: 1.1.3

Extensions loaded successfully


In [3]:
# Second cell - Configure S3/MinIO settings
def configure_s3(conn, access_key="admin", secret_key="password", 
                endpoint="minio:9000", region="us-east-1", use_ssl=False):
    conn.execute(f"""
    SET s3_region='{region}';
    SET s3_access_key_id='{access_key}';
    SET s3_secret_access_key='{secret_key}';
    SET s3_endpoint='{endpoint}';
    SET s3_use_ssl={str(use_ssl).lower()};
    """)
    print("S3/MinIO configurations set.")

# Configure S3
configure_s3(conn)


S3/MinIO configurations set.


In [4]:

# Third cell - Function to find latest metadata file
def find_latest_metadata(conn, iceberg_table_path):
    """Find the latest metadata file for an Iceberg table"""
    try:
        # List all metadata files
        print("Searching for metadata files...")
        metadata_files = conn.execute(f"""
            SELECT *
            FROM glob('{iceberg_table_path}/metadata/*.json')
            ORDER BY file DESC
        """).fetchdf()
        
        if metadata_files.empty:
            raise Exception("No metadata files found")
        
        print(f"\nFound {len(metadata_files)} metadata files:")
        print(metadata_files)
        
        # Extract metadata file numbers
        def extract_number(filename):
            match = re.search(r'/(\d{5})-', filename)
            return int(match.group(1)) if match else -1
        
        metadata_files['number'] = metadata_files['file'].apply(extract_number)
        latest_metadata = metadata_files.loc[metadata_files['number'].idxmax(), 'file']
        
        print(f"\nLatest metadata file: {latest_metadata}")
        return latest_metadata
        
    except Exception as e:
        print(f"Error finding latest metadata: {str(e)}")
        return None


In [5]:

# Fourth cell - Find latest metadata for your table
iceberg_table_path = "s3://warehouse/nyc/taxis"
latest_metadata = find_latest_metadata(conn, iceberg_table_path)


Searching for metadata files...

Found 39 metadata files:
                                                 file
0   s3://warehouse/nyc/taxis/metadata/00024-41849b...
1   s3://warehouse/nyc/taxis/metadata/00023-9258d4...
2   s3://warehouse/nyc/taxis/metadata/00022-6d3ab8...
3   s3://warehouse/nyc/taxis/metadata/00021-b58fd1...
4   s3://warehouse/nyc/taxis/metadata/00020-d39091...
5   s3://warehouse/nyc/taxis/metadata/00019-853d76...
6   s3://warehouse/nyc/taxis/metadata/00018-886824...
7   s3://warehouse/nyc/taxis/metadata/00017-43b584...
8   s3://warehouse/nyc/taxis/metadata/00016-71338f...
9   s3://warehouse/nyc/taxis/metadata/00015-fd607c...
10  s3://warehouse/nyc/taxis/metadata/00014-dd3476...
11  s3://warehouse/nyc/taxis/metadata/00013-0f0a0a...
12  s3://warehouse/nyc/taxis/metadata/00012-582c06...
13  s3://warehouse/nyc/taxis/metadata/00012-4c4b3a...
14  s3://warehouse/nyc/taxis/metadata/00011-d66f07...
15  s3://warehouse/nyc/taxis/metadata/00011-ca8992...
16  s3://warehouse/nyc/t

In [6]:

# Cell 4: Run simple query
def run_simple_query(conn, metadata_file):
    """Run and time a simple aggregation query"""
    query = """
        SELECT
            COUNT(*) AS total_records,
            AVG(trip_distance) AS avg_trip_distance,
            MAX(total_amount) AS max_total_amount,
            MIN(fare_amount) AS min_fare_amount
        FROM iceberg_scan('{metadata_file}')
    """
    
    print("Executing simple aggregation query...")
    print("\nQuery:")
    print(query.format(metadata_file=metadata_file))
    
    start_time = time.time()
    result = conn.execute(query.format(metadata_file=metadata_file)).fetchdf()
    duration = time.time() - start_time
    
    print(f"\nQuery completed in {duration:.2f} seconds")
    print("\nResults:")
    print(result)
    
    return {"duration": duration, "result": result}

# Run simple query
simple_results = run_simple_query(conn, latest_metadata)


Executing simple aggregation query...

Query:

        SELECT
            COUNT(*) AS total_records,
            AVG(trip_distance) AS avg_trip_distance,
            MAX(total_amount) AS max_total_amount,
            MIN(fare_amount) AS min_fare_amount
        FROM iceberg_scan('s3://warehouse/nyc/taxis/metadata/00024-41849b29-2da8-432a-9f14-0acee4f595b0.metadata.json')
    


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Query completed in 4.42 seconds

Results:
   total_records  avg_trip_distance  max_total_amount  min_fare_amount
0       77966324           5.040317         401095.62     -133391414.0


In [7]:
# Cell 5: Run complex query
def run_complex_query(conn, metadata_file):
    """Run and time a complex analysis query"""
    query = """
        SELECT
            payment_type,
            COUNT(*) AS trip_count,
            AVG(total_amount) AS avg_total_amount,
            MAX(tip_amount) AS max_tip_amount,
            SUM(CASE WHEN passenger_count > 1 THEN 1 ELSE 0 END) as shared_rides
        FROM iceberg_scan('{metadata_file}')
        WHERE trip_distance > 2 AND total_amount > 0
        GROUP BY payment_type
        ORDER BY trip_count DESC
    """
    
    print("Executing complex analysis query...")
    print("\nQuery:")
    print(query.format(metadata_file=metadata_file))
    
    start_time = time.time()
    result = conn.execute(query.format(metadata_file=metadata_file)).fetchdf()
    duration = time.time() - start_time
    
    print(f"\nQuery completed in {duration:.2f} seconds")
    print("\nResults:")
    print(result)
    
    return {"duration": duration, "result": result}

# Run complex query
complex_results = run_complex_query(conn, latest_metadata)

Executing complex analysis query...

Query:

        SELECT
            payment_type,
            COUNT(*) AS trip_count,
            AVG(total_amount) AS avg_total_amount,
            MAX(tip_amount) AS max_tip_amount,
            SUM(CASE WHEN passenger_count > 1 THEN 1 ELSE 0 END) as shared_rides
        FROM iceberg_scan('s3://warehouse/nyc/taxis/metadata/00024-41849b29-2da8-432a-9f14-0acee4f595b0.metadata.json')
        WHERE trip_distance > 2 AND total_amount > 0
        GROUP BY payment_type
        ORDER BY trip_count DESC
    


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Query completed in 6.99 seconds

Results:
   payment_type  trip_count  avg_total_amount  max_tip_amount  shared_rides
0             1    27504582         37.871267         1400.16     6696053.0
1             2     5922309         33.125444           32.00     1745599.0
2             0     1603076         35.306151           92.35           0.0
3             4      134663         42.777958           95.00       32343.0
4             3       90810         33.755191           90.00       20430.0


In [8]:
def run_advanced_analysis_duckdb(conn, metadata_file):
    """Run advanced fare analysis with trend detection and interpolation for DuckDB
    
    Args:
        conn: DuckDB connection
        metadata_file: Path to Iceberg metadata file
    Returns:
        dict: Contains duration and results dataframe
    """
    query = """
    WITH fare_stats AS (
        SELECT 
            date_trunc('month', tpep_pickup_datetime) as month,
            payment_type,
            AVG(fare_amount) as avg_base_fare,
            AVG(total_amount) as avg_total_fare,
            AVG(tip_amount) as avg_tip,
            COUNT(*) as num_rides,
            STDDEV(fare_amount) as fare_stddev
        FROM iceberg_scan('{metadata_file}')
        WHERE fare_amount > 0 
        AND total_amount > 0
        AND EXTRACT(year FROM tpep_pickup_datetime) >= 2022
        GROUP BY 
            date_trunc('month', tpep_pickup_datetime),
            payment_type
    ),
    interpolated_points AS (
        SELECT 
            month,
            payment_type,
            avg_base_fare,
            avg_total_fare,
            CASE 
                WHEN avg_tip IS NULL THEN 
                    LAG(avg_tip, 1) OVER (PARTITION BY payment_type ORDER BY month) +
                    (LEAD(avg_tip, 1) OVER (PARTITION BY payment_type ORDER BY month) -
                     LAG(avg_tip, 1) OVER (PARTITION BY payment_type ORDER BY month)) * 0.5
                ELSE avg_tip
            END as interpolated_tip,
            num_rides,
            fare_stddev,
            avg_base_fare - LAG(avg_base_fare) OVER (PARTITION BY payment_type ORDER BY month) as fare_change,
            100.0 * (avg_base_fare - LAG(avg_base_fare) OVER (PARTITION BY payment_type ORDER BY month)) / 
                NULLIF(LAG(avg_base_fare) OVER (PARTITION BY payment_type ORDER BY month), 0) as fare_change_pct
        FROM fare_stats
    )
    SELECT 
        month,
        payment_type,
        ROUND(avg_base_fare, 2) as avg_base_fare,
        ROUND(avg_total_fare, 2) as avg_total_fare,
        ROUND(interpolated_tip, 2) as interpolated_tip,
        num_rides,
        ROUND(fare_stddev, 2) as fare_stddev,
        ROUND(fare_change, 2) as fare_change,
        ROUND(fare_change_pct, 1) as fare_change_pct,
        CASE 
            WHEN fare_change_pct > 5 THEN 'Strong Increase'
            WHEN fare_change_pct BETWEEN 1 AND 5 THEN 'Moderate Increase'
            WHEN fare_change_pct BETWEEN -1 AND 1 THEN 'Stable'
            WHEN fare_change_pct BETWEEN -5 AND -1 THEN 'Moderate Decrease'
            ELSE 'Strong Decrease'
        END as trend_category
    FROM interpolated_points
    ORDER BY month DESC, payment_type;
    """
    
    print("Executing advanced fare trend analysis (DuckDB)...")
    start_time = time.time()
    
    try:
        result = conn.execute(query.format(metadata_file=metadata_file)).fetchdf()
        duration = time.time() - start_time
        
        print(f"\nAnalysis completed in {duration:.2f} seconds")
        print(f"Total records analyzed: {len(result):,}")
        print("\nResults preview (most recent month):")
        print(result.head())
        
        # Calculate summary statistics
        print("\nSummary Statistics:")
        print(f"Average base fare: ${result['avg_base_fare'].mean():.2f}")
        print(f"Average total fare: ${result['avg_total_fare'].mean():.2f}")
        print(f"Average tip: ${result['interpolated_tip'].mean():.2f}")
        print("\nTrend Distribution:")
        print(result['trend_category'].value_counts())
        
        return {"duration": duration, "result": result}
    
    except Exception as e:
        print(f"Error executing DuckDB analysis: {str(e)}")
        return None

# Run advanced analysis
advanced_results = run_advanced_analysis_duckdb(conn, latest_metadata)



Executing advanced fare trend analysis (DuckDB)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Analysis completed in 13.01 seconds
Total records analyzed: 125

Results preview (most recent month):
                      month  payment_type  avg_base_fare  avg_total_fare  \
0 2024-01-01 00:00:00+00:00             1          72.96           85.72   
1 2024-01-01 00:00:00+00:00             2           6.50           11.50   
2 2023-12-01 00:00:00+00:00             0          22.90           28.99   
3 2023-12-01 00:00:00+00:00             1          20.02           30.12   
4 2023-12-01 00:00:00+00:00             2          20.10           25.41   

   interpolated_tip  num_rides  fare_stddev  fare_change  fare_change_pct  \
0              8.67          5        40.06        52.94            264.4   
1              0.00          1          NaN       -13.60            -67.7   
2              1.77     176966        14.13         1.11              5.1   
3              4.49    2574324        17.90        -0.04             -0.2   
4              0.00     536489        19.89         0.1

In [11]:
print(advanced_results)

{'duration': 13.014678716659546, 'result':                         month  payment_type  avg_base_fare  avg_total_fare  \
0   2024-01-01 00:00:00+00:00             1          72.96           85.72   
1   2024-01-01 00:00:00+00:00             2           6.50           11.50   
2   2023-12-01 00:00:00+00:00             0          22.90           28.99   
3   2023-12-01 00:00:00+00:00             1          20.02           30.12   
4   2023-12-01 00:00:00+00:00             2          20.10           25.41   
..                        ...           ...            ...             ...   
120 2022-01-01 00:00:00+00:00             1          12.77           19.73   
121 2022-01-01 00:00:00+00:00             2          12.83           16.53   
122 2022-01-01 00:00:00+00:00             3          11.97           15.45   
123 2022-01-01 00:00:00+00:00             4         138.37          141.96   
124 2022-01-01 00:00:00+00:00             5           8.50           11.80   

     interpolated_ti

In [12]:
# Cell 6: Save benchmark results
result = BenchmarkResult("DuckDB", duckdb.__version__)

# For simple query:
total_records = simple_results["result"]["total_records"].iloc[0]  # Get the COUNT(*) result
result.add_query_result(
    "simple_aggregation",
    simple_results["duration"],
    total_records,
    simple_results["result"]
)

# For complex query:
total_records = complex_results["result"]["trip_count"].sum()  # Sum all group counts
result.add_query_result(
    "complex_analysis",
    complex_results["duration"],
    total_records,
    complex_results["result"]
)

result.add_query_result(
    "advanced_analysis",
    advanced_results["duration"],
    advanced_results["result"]["num_rides"].sum(),
    advanced_results["result"]
)

# Save results (automatically captures resource usage)
result.save('duckdb_benchmark_results.json')

Benchmark results saved to duckdb_benchmark_results.json


In [13]:
# Cell 7: Clean up
conn.close()
print("\nDuckDB connection closed")


DuckDB connection closed
